Load all the dependencies

In [1]:
from cuml.dask.cluster import KMeans
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait
import dask.array as da
import time
import os 
import pickle
from cuml.dask.datasets import make_blobs


Set up useful environmental variable.

Experiment with the flags, for example enable ucx. What happens of we do not enable ucx?


In [2]:
os.environ["DASK_DISTRIBUTED__COMM__UCX__NVLINK"] = "True"
os.environ["DASK_DISTRIBUTED__COMM__UCX__INFINIBAND"] = "True"
os.environ["DASK_DISTRIBUTED__COMM__UCX__NET_DEVICES"] = "ib0"
os.environ["DASK_DISTRIBUTED__COMM__UCX__CREATE_CUDA_CONTEXT"]="True"
os.environ["DASK_DISTRIBUTED__COMM__UCX__CUDA_COPY"]="True"
os.environ["DASK_DISTRIBUTED__COMM__UCX__TCP"]="True"
os.environ["DASK_DISTRIBUTED__COMM__UCX__NVLINK"]="True"
os.environ["DASK_DISTRIBUTED__COMM__UCX__INFINIBAND"]="True"
os.environ["DASK_DISTRIBUTED__COMM__UCX__RDMACM"]="True"
os.environ["UCX_MEMTYPE_REG_WHOLE_ALLOC_TYPES"]="cuda"
os.environ["UCX_MEMTYPE_CACHE"]="n"
current_dir = os.environ.get('SLURM_SUBMIT_DIR')


Set up a LocalCUDACluster with 4 workers supposed to run on 4 GPUs in a single node


In [3]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=[0, 1, 2, 3],
                               n_workers=4,
                               threads_per_worker=8,
                               protocol="ucx",
                               interface="ib0",
                               enable_tcp_over_ucx=True,
                               enable_infiniband=True,
                               enable_nvlink=True,
                               enable_rdmacm=True,
                               rmm_pool_size="50GB",)

client = Client(cluster)

client.wait_for_workers(4)
client

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://10.128.44.33:8787/status,
Dashboard: http://10.128.44.33:8787/status,Workers: 4
Total threads: 32,Total memory: 482.42 GiB
Status: running,Using processes: True
Comm: ucx://10.128.44.33:33002,Workers: 4
Dashboard: http://10.128.44.33:8787/status,Total threads: 32
Started: Just now,Total memory: 482.42 GiB
Comm: ucx://10.128.44.33:37450,Total threads: 8
Dashboard: http://10.128.44.33:38717/status,Memory: 120.61 GiB
Nanny: ucx://10.128.44.33:53420,


In [4]:
# Set up the number of samples
n_samples = 200000000
# Specify further informations for the dataset we are going to generate
n_features = 6
start_random_state = 170
n_parts = 20
n_clusters = 8


Get the dataset (X, y), together with the true centers of the blobs dataset


In [5]:
X, _, centers = make_blobs(n_samples=n_samples, n_features=n_features, centers=n_clusters, cluster_std=0.4, 
                               random_state=start_random_state, n_parts=n_parts, return_centers=True)

In [6]:
X


dask.array<concatenate, shape=(200000000, 6), dtype=float32, chunksize=(10000000, 6), chunktype=cupy.ndarray>

Define the KMeans model and fit over the data

Use the KMeans from cuml.dask.cluster with n_clusters defined above 


In [7]:
kmeans_model = # TODO


SyntaxError: invalid syntax (2194383097.py, line 1)

Fit kmeans_model over the blobs data

In [ ]:
# TODO

Compute the score over the dataset


In [ ]:
score = # TODO

score


The distributed estimator wrappers inside of the cuml.dask are not intended to be pickled directly. 

The Dask cuML estimators provide a function get_combined_model(), which returns the trained single-GPU model for pickling. 


In [ ]:
single_gpu_model = # TODO


Save the model to file


In [ ]:
pickle.dump(single_gpu_model,  open(current_dir + "/kmeans_model.pkl", "wb"))


Load the model from file

In [ ]:
single_gpu_model = pickle.load(open(current_dir + "/kmeans_model.pkl", "rb"))


Check the centers

In [ ]:
print("\nsingle_gpu_model.cluster_centers_:\n",  single_gpu_model.cluster_centers_, flush=True)
print("\nTrue centers:\n", centers, flush=True)


Closing...

In [ ]:
client.close()
